In [15]:
import os
import pandas as pd
import numpy as np

In [16]:
!mkdir -p data/
if not os.path.exists('data/rankings_1973-2017.csv'):
    !curl -sSL -o data/rankings_1973-2017.csv https://datahub.io/sports-data/atp-world-tour-tennis-data/r/rankings_1973-2017.csv

atp_rankings = pd.read_csv('data/rankings_1973-2017.csv')

In [17]:
def find_all_years(df):
    all_years = df['week_year'].unique()
    all_years.sort()
    all_years_list = all_years.tolist()
    return all_years_list

def find_last_date(df, year):
    search_year = df[df['week_year'] == year]
    search_month = sorted(search_year['week_month'].unique())[-1]
    search_year_search_month = search_year[search_year['week_month'] == search_month]
    search_day = sorted(search_year_search_month['week_day'].unique())[-1]
    return search_month, search_day

def year_end_top_100(df, year):
    last_month, last_day = find_last_date(df, year)
    search_year = df[df['week_year'] == year]
    year_end_month = search_year[search_year['week_month'] == last_month]
    year_end_day = year_end_month[year_end_month['week_day'] == last_day]
    sorted_top = year_end_day.sort_values(by=['rank_number'])
    top_100 = sorted_top[:100]
    age_top_100 = top_100['player_age'].values
    age_top_100_list = age_top_100.tolist()
    return age_top_100_list

def year_end_rank(df, year):
    last_month, last_day = find_last_date(df, year)
    search_year = df[df['week_year'] == year]
    year_end_month = search_year[search_year['week_month'] == last_month]
    year_end_day = year_end_month[year_end_month['week_day'] == last_day]
    sorted_top = year_end_day.sort_values(by=['rank_number'])
    return sorted_top

def find_youngest_in_top_100(df, year):
    last_month, last_day = find_last_date(df, year)
    search_year = df[df['week_year'] == year]
    year_end_month = search_year[search_year['week_month'] == last_month]
    year_end_day = year_end_month[year_end_month['week_day'] == last_day]
    sorted_top = year_end_day.sort_values(by=['rank_number'])
    top_100 = sorted_top[:100]
    find_youngest = sorted(top_100['player_age'].unique())[0]
    return find_youngest

def find_oldest_in_top_100(df, year):
    last_month, last_day = find_last_date(df, year)
    search_year = df[df['week_year'] == year]
    year_end_month = search_year[search_year['week_month'] == last_month]
    year_end_day = year_end_month[year_end_month['week_day'] == last_day]
    sorted_top = year_end_day.sort_values(by=['rank_number'])
    top_100 = sorted_top[:100]
    find_oldest = sorted(top_100['player_age'].unique())[-1]
    return find_oldest

In [ ]:
averages_for_every_year = []
for year in find_all_years(atp_rankings):
   average_100 = np.mean(year_end_top_100(atp_rankings, year))
   averages_for_every_year.append(average_100)

youngest_for_every_year = []
for year in find_all_years(atp_rankings):
   youngest = find_youngest_in_top_100(atp_rankings, year)
   youngest_for_every_year.append(youngest)

oldest_for_every_year = []
for year in find_all_years(atp_rankings):
   oldest = find_oldest_in_top_100(atp_rankings, year)
   oldest_for_every_year.append(oldest)

In [18]:
print(find_oldest_in_top_100(atp_rankings, 2017))

In [ ]:
year_1976 = year_end_top_100(atp_rankings, 1976) #problem: 'nan' instead of 23
year_end_1980 = year_end_rank(atp_rankings, 1980) #problem: only two rows